In [21]:
using BenchmarkTools
using Base: tail
println("Julia version $VERSION")

Julia version 1.7.0


In [22]:
function f(iter)
    s = 0
    for I in iter
        s += first(I)
    end
    s
end

f (generic function with 1 method)

In [24]:
doubtup(xs...) = (xs,xs);

In [83]:
symgrp_size(Nt,Nsym) = Nsym > 0 ? binomial(Nt-1+Nsym, Nsym) : binomial(Nt, -Nsym)
struct SymIndexIter{Nsym}
    size::Int
end
Base.IteratorSize(::Type{<:SymIndexIter}) = Base.HasLength()
Base.IteratorEltype(::Type{<:SymIndexIter}) = Base.HasEltype()
Base.ndims(::SymIndexIter{Nsym}) where Nsym = Nsym
Base.eltype(::Type{SymIndexIter{Nsym}}) where Nsym = NTuple{Nsym,Int}
Base.length(iter::SymIndexIter{Nsym}) where Nsym = symgrp_size(iter.size,Nsym)
Base.first(iter::SymIndexIter{Nsym}) where Nsym = Nsym > 0 ? ntuple(one,Val(Nsym)) : ntuple(i->i,Val(-Nsym))
Base.last(iter::SymIndexIter{Nsym}) where Nsym = Nsym>0 ? ntuple(i->iter.size,Val(Nsym)) : ntuple(i->iter.size+Nsym+i #= Nsym < 0 here =#,Val(-Nsym))

@inline function Base.iterate(iter::SymIndexIter)
    I = first(iter)
    I, I
end

function itercode(Nsym)
    newstate = Any[:(state[$i]) for i=1:abs(Nsym)]
    code = Expr(:block)
    # global dimension index
    for d = 1:abs(Nsym)
        maxv = d==abs(Nsym) ? :(iter.size) : (Nsym>0 ? :(state[$(d+1)]) : :(state[$(d+1)]-1))
        newstate[d] = :( state[$d] + 1 )
        push!(code.args, :( state[$d] < $maxv && return doubtup($(newstate...)) ))
        newstate[d] = Nsym>0 ? 1 : d
    end
    push!(code.args, :( return nothing ))
    code
end
@inline @generated Base.iterate(iter::SymIndexIter{Nsym}, state) where {Nsym} = itercode(Nsym)

In [88]:
@inline function Base.iterate(iter::SymIndexIter{2}, state)
    state[1] < state[2]  && return doubtup(state[1]+1, state[2])
    state[2] < iter.size && return doubtup(1,state[2]+1)
    return nothing
end

@btime f(it) setup=(it=SymIndexIter{2}(50))

  769.963 ns (0 allocations: 0 bytes)


22100

In [92]:
@inline @generated Base.iterate(iter::SymIndexIter{2}, state) = itercode(2)

@btime f(it) setup=(it=SymIndexIter{2}(50))

  772.185 ns (0 allocations: 0 bytes)


22100

In [87]:
function Base.iterate(iter::SymIndexIter{2}, state)
    i1,i2 = state
    d, i1 = divrem(i1,i2)
    I = (i1+1,i2+d)
    ifelse(I[2]<=iter.size, (I,I), nothing)
end

@btime f(it) setup=(it=SymIndexIter{2}(50))

  13.839 μs (0 allocations: 0 bytes)


22100

In [103]:
struct SymIndexIter2{Nsym}
    size::Int
    "create an iterator that gives i1<=i2<=i3 etc for one index group"
    SymIndexIter2(Nsym,size) = new{Nsym}(size)
end
Base.IteratorSize(::Type{<:SymIndexIter2}) = Base.HasLength()
Base.IteratorEltype(::Type{<:SymIndexIter2}) = Base.HasEltype()
Base.ndims(::SymIndexIter2{Nsym}) where Nsym = Nsym
Base.eltype(::Type{SymIndexIter2{Nsym}}) where Nsym = NTuple{Nsym,Int}
Base.length(iter::SymIndexIter2{Nsym}) where Nsym = symgrp_size(iter.size,Nsym)
Base.first(iter::SymIndexIter2{Nsym}) where Nsym = Nsym > 0 ? ntuple(one,Val(Nsym)) : ntuple(i->i,Val(-Nsym))
Base.last(iter::SymIndexIter2{Nsym}) where Nsym = Nsym>0 ? ntuple(i->iter.size,Val(Nsym)) : ntuple(i->iter.size+Nsym+i #= Nsym < 0 here =#,Val(-Nsym))

@inline function Base.iterate(iter::SymIndexIter2)
    I = first(iter)
    I, I
end
@inline function Base.iterate(iter::SymIndexIter2, state)
    valid, I = __inc(iter, state)
    valid ? (I, I) : nothing
end

@inline function __inc(iter::SymIndexIter2{Nsym}, state::NTuple{N,Int}) where {Nsym,N}
    if state[1] + (Nsym<0) < state[2]
        return true, (state[1]+1, tail(state)...)
    end
    valid, I = __inc(iter, tail(state))
    # if Nsym<0, lowest possible value for nth index is n
    In = Nsym>0 ? 1 : (1 - Nsym - N)
    return valid, (In, I...)
end
@inline function __inc(iter::SymIndexIter2, state::Tuple{Int})
    # last index can go until iter.size independent of (anti-)symmetry
    state[1] < iter.size, (state[1]+1,)
end

@benchmark f(it) setup=(it=SymIndexIter2(2,50))

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.942 μs …   4.128 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.038 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.065 μs ± 153.358 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂ ▁▂  ▄▅▆██▅  ▂▃▃▅▅▂                                        ▂
  █▇██▁▁██████▄▃██████▁▁▁▁▁▁▁▁▃▁▃▁▃▁▁▃▅▅▆▇▇▇▇▇▇▆▆▄▆▄▅▅▅▅▄▅▄▄▃ █
  1.94 μs      Histogram: log(frequency) by time      2.51 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.